In [ ]:
pip install azure-storage-file-datalake

Python interpreter will be restarted.
Python interpreter will be restarted.


In [ ]:
%run "/Shared/Common Utilities/helper_utils_Indrani"

In [ ]:
config = get_config()

In [ ]:
###Read Populations file from ContainerBlob
file_location = "wasbs://wastewater@cctdash0000prd0504blob.blob.core.windows.net/ontario___extended_aggregated_wsi_dataset.csv"
WasteW_count_major = spark.read.options(header=True, inferSchema=True).csv(file_location)
WasteW_count_major.createOrReplaceTempView("WasteW_count_major")
#sqlDF = spark.sql("SELECT count(*) FROM WasteW_count_major")
WasteW_refine = WasteW_count_major.select("siteID","sewershedPop")
#display(WasteW_refine)

In [ ]:
WasteW_refine = WasteW_refine.select("siteID","sewershedPop").distinct()
# display(WasteW_refine)

In [ ]:
import datetime
date = datetime.date.today()

In [ ]:
###read the LatestTrends combined for comparison
#####creating path, pulling date(latest tuesday), appending to the existing wasbs@container

from datetime import datetime, timedelta
Pres = date.today()
#datenow = datetime.utcnow().strftime("-%Y-%m-%a")
offset = (Pres.weekday() - 1) % 7
Date_T = str(Pres - timedelta(days=offset))
address = "wasbs://wastewater@cctdash0000prd0504blob.blob.core.windows.net/LATEST_TREND_COMBINED-"
format = ".csv"
path = address+Date_T+format
print(path)

wasbs://wastewater@cctdash0000prd0504blob.blob.core.windows.net/LATEST_TREND_COMBINED-2023-05-30.csv


In [ ]:
#### Reading the Path for the Tuesday upload of wastewater

filelocation = path
try:
    FileTuesdayWW = spark.read.format("csv").options(header=True, inferSchema=False).load  (filelocation)
#display(FileTuesdayWW.withColumn("filename",input_file_name()))
    display("File loaded")
except Exception as e:
    print("No new file exists")
    #e.getMessage()

'File loaded'

In [ ]:
###read the Tuesday LatestTrends combined for comparison
WasteW_latesttrends = spark.read.options(header=True, inferSchema=True).csv(filelocation)
WasteW_latesttrends.createOrReplaceTempView("WasteW_latesttrends")
WasteW_latesttrends=WasteW_latesttrends.na.replace("NA","Null")
wwlatesttrends = spark.sql("SELECT count(*) FROM WasteW_latesttrends")
wwlatesttrends.show()

In [ ]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import lit, when

#### joining LATEST_TREND file and ontario___extended_aggregated_wsi_dataset.csv by SITEID & POPULATION('sewershedPop')
combined_LATEST_TREND = WasteW_latesttrends.join(WasteW_refine,['siteID'],how='inner')
#display(combined_LATEST_TREND)

####calling the New PHUnames.csv files stored in dbfs:/FileStore/wastewater/PHUNAMes.csv
# df_PHUnames = spark.read.format("csv").option("header","true").load('dbfs:/FileStore/wastewater/PHUNames.csv')
# combined_LATEST_TREND = combined_LATEST_TREND.join(df_PHUnames,['sys_PHU'],how='inner')
# combined_LATEST_TREND = combined_LATEST_TREND.drop(combined_LATEST_TREND['sys_PHU'])   

###ETL based on Increasing, Decreasing or NoChange
combined_LATEST_TREND = combined_LATEST_TREND.withColumn("Status", when((combined_LATEST_TREND.wwSlope>10) & \
                             (combined_LATEST_TREND.wwSignificance=="Yes"), "Increasing")
                             .when((combined_LATEST_TREND.wwSlope<=-10) & \
                             (combined_LATEST_TREND.wwSignificance=="Yes"), "Decreasing")
                             .when((combined_LATEST_TREND.wwSlope>-10) | \
                             (combined_LATEST_TREND.wwSignificance=="No"), "Little/No Change")
                             .otherwise("No Trend Available"))

##write transformed file back to blobContainer@wastewater
combined_LATEST_TREND
file_location_output = "wasbs://wastewater@cctdash0000prd0504blob.blob.core.windows.net/output"
combined_LATEST_TREND.coalesce(1).write.mode("overwrite").options(header=True).format("csv").save(file_location_output)

#####write Table to Public/ist_0504_cctdash_0000_uc
wastewater_output = config["cct_db_schema"] + ".wastewater_output"
spark.sql(f"DROP TABLE IF EXISTS {wastewater_output}")
combined_LATEST_TREND.write.saveAsTable(wastewater_output)

In [ ]:
######################################################################################
#############rename part-00000* file to wastewater_output.csv

targetname = ''
from azure.storage.filedatalake import FileSystemClient
file_system = FileSystemClient.from_connection_string(ConnectionString,file_system_name="wastewater")

paths = file_system.get_paths()
for path in paths:
    #print(path.name + '\n')
    if path.name.startswith('output/part-0000'):
        targetname = path.name
        
#print(targetname)
fileloc_RenameFile = "wasbs://wastewater@cctdash0000prd0504blob.blob.core.windows.net/"+targetname
old_name = fileloc_RenameFile
new_name = r"wasbs://wastewater@cctdash0000prd0504blob.blob.core.windows.net/wastewater_output.csv"

dbutils.fs.mv(old_name, new_name, True)

PATH = "wasbs://wastewater@cctdash0000prd0504blob.blob.core.windows.net/output/"
for i in dbutils.fs.ls(PATH):
    dbutils.fs.rm(i[0],True)

output = 'output'
PATH = "wasbs://wastewater@cctdash0000prd0504blob.blob.core.windows.net/"+output
dbutils.fs.rm(PATH,True)

Out[10]: True

In [ ]:
from pyspark.sql.functions import countDistinct, sum, avg, concat, col, lit
from pyspark.sql import functions as B

count_sewersheds = combined_LATEST_TREND.groupBy("sys_PHU","sys_PHU_region").agg(countDistinct("sys_sewershedName"). \
                   alias("Total_sewersheds_count"))
count_sewersheds_N = combined_LATEST_TREND.withColumn("Status_N", when((combined_LATEST_TREND.Status == 'Increasing'), "1"))
_fil_count_sewersheds_N = count_sewersheds_N.select("sys_PHU","Status_N").filter(col("Status_N").isNotNull())
combined_count_sewersheds = count_sewersheds.join(_fil_count_sewersheds_N,['sys_PHU'],how='inner')
fil_count_sewersheds_N = combined_count_sewersheds.groupBy("sys_PHU").count()
final_count_sewershedPHU_Region = count_sewersheds.join(fil_count_sewersheds_N,['sys_PHU'],how='inner').drop("Status_N").distinct(). \
                                withColumnRenamed("count", "Status_Increasing")

final_count_sewershedPHU_Region = final_count_sewershedPHU_Region.withColumn("Region", lit("Region"))
                                  
# display(final_count_sewershedPHU_Region)

final_count_sewershedPHU_Region = final_count_sewershedPHU_Region.withColumn('PHU_region', B.concat(B.col('sys_PHU_region'), B.lit(" Region")))
# final_count_sewershedPHU_Region = final_count_sewershedPHU_Region.withColumn('Status_Increasing', B.concat(B.col('Status_Increasing'), B.lit(" of")))


final_count_sewershedPHU_Region = final_count_sewershedPHU_Region.drop(final_count_sewershedPHU_Region['sys_PHU_region']) \
                                                                 .drop(final_count_sewershedPHU_Region['Region']) 

display(final_count_sewershedPHU_Region)    #sys_PHU_region, Total_sewersheds_count,Region, Sewershed

##write transformed file - PHU_increase_report.csv back to blobContainer@wastewater
file_location_output = "wasbs://wastewater@cctdash0000prd0504blob.blob.core.windows.net/output"
final_count_sewershedPHU_Region.coalesce(1).write.mode("overwrite").options(header=True).format("csv").save(file_location_output)

#####write Table to Public/ist_0504_cctdash_0000_uc
finalcountSW_PHU_Region = config["cct_db_schema"] + ".PHU_increase_report"
spark.sql(f"DROP TABLE IF EXISTS {finalcountSW_PHU_Region}")
final_count_sewershedPHU_Region.write.saveAsTable(finalcountSW_PHU_Region)

In [ ]:
######################################################################################
#############rename part-00000* file to PHU_increase_report.csv

targetname = ''
from azure.storage.filedatalake import FileSystemClient
file_system = FileSystemClient.from_connection_string(ConnectionString,file_system_name="wastewater")

paths = file_system.get_paths()
for path in paths:
    #print(path.name + '\n')
    if path.name.startswith('output/part-0000'):
        targetname = path.name
        
#print(targetname)
fileloc_RenameFile = "wasbs://wastewater@cctdash0000prd0504blob.blob.core.windows.net/"+targetname
old_name = fileloc_RenameFile
new_name = r"wasbs://wastewater@cctdash0000prd0504blob.blob.core.windows.net/PHU_increase_report.csv"

dbutils.fs.mv(old_name, new_name, True)

PATH = "wasbs://wastewater@cctdash0000prd0504blob.blob.core.windows.net/output/"
for i in dbutils.fs.ls(PATH):
    dbutils.fs.rm(i[0],True)

output = 'output'
PATH = "wasbs://wastewater@cctdash0000prd0504blob.blob.core.windows.net/"+output
dbutils.fs.rm(PATH,True)

print("Reports uploaded to blob successfully")

Reports uploaded to blob successfully
